In [216]:
import torch
import numpy as np
import scipy
import math

USE_CUDA = torch.cuda.is_available() 
print(f"cuda available: {USE_CUDA}")

device = torch.device('cuda:1' if USE_CUDA else 'cpu') 
print(f"code run on device:: {device}")

cuda available: True
code run on device:: cuda:1


# 1. verification of simplication

In [3]:
onePhioneonePsione = 10
onePhiPsiOne = 2
trPhiPsi = 3
n = 20

print(
          4 * (onePhioneonePsione - 4 * onePhiPsiOne + 2 * trPhiPsi)
        - 8 * (n-3) *(onePhiPsiOne - trPhiPsi)
        + 4 * (n-3)*(n-2) * trPhiPsi
        )

print(
          4 * onePhioneonePsione
        - 8 * (n-1) * onePhiPsiOne
        + 4 * (n-1) * (n-2) * trPhiPsi
        )

3840
3840


# 2. verification of tr(phi psi)

In [5]:
data_X = torch.normal(0,1, [5000,9]).to(device)
data_Y = torch.normal(0,1, [5000,9]).to(device)

In [6]:
#einsum verification
# XtX
Phi_einsum = torch.einsum('ji,jk->ik', data_X, data_X)
Phi_matmul = torch.matmul(torch.transpose(data_X, 0, 1), data_X)
Phi_einsum - Phi_matmul
#success


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:1')

In [9]:
#einsum verification
# XXt
Phi_einsum = torch.einsum('ij,kj->ik', data_X, data_X)
Phi_matmul = torch.matmul(data_X, torch.transpose(data_X, 0, 1))
Phi_einsum - Phi_matmul
#success

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:1')

In [2]:
# transpose verification
data_X = torch.normal(0,1, [5000,9]).to(device)
data_Y = torch.normal(0,1, [5000,9]).to(device)

Phi = torch.einsum('ij,kj->ik', data_X, data_X)
Psi = torch.einsum('ij,kj->ik', data_Y, data_Y)
tr_matmul = torch.trace(torch.matmul(Phi, Psi))
tr_sumsq = torch.sum(
    torch.square(
        torch.einsum('ji,jk->ik', data_X, data_Y)
    )
)
print(tr_matmul)
print(tr_sumsq)

tensor(301518.5625, device='cuda:1')
tensor(301518.5625, device='cuda:1')


In [6]:
# transpose verification on large size
data_X = torch.normal(0,1, [50000,9]).to(device)
data_Y = torch.normal(0,1, [50000,9]).to(device)


tr_sumsq = torch.sum(
    torch.square(
        torch.einsum('ji,jk->ik', data_X, data_Y)
    )
)
print(tr_sumsq)

tensor(4756153., device='cuda:1')


In [12]:
# diag Phi verification
data_X = torch.normal(0,1, [5000,9]).to(device)

Phi = torch.einsum('ij,kj->ik', data_X, data_X)
Phi_diag_matmul = torch.diagonal(Phi)

Phi_diag_sumsq = torch.sum( torch.square(data_X), axis = 1)
print(Phi_diag_matmul - Phi_diag_sumsq)
print(Phi_diag_matmul)
print(Phi_diag_sumsq)

#success

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.9073e-06, 0.0000e+00,
        4.7684e-07], device='cuda:1')
tensor([10.3022, 11.7083,  4.3699,  ..., 24.5278,  6.1498,  5.9414],
       device='cuda:1')
tensor([10.3022, 11.7083,  4.3699,  ..., 24.5278,  6.1498,  5.9414],
       device='cuda:1')


In [34]:
# tr Phi diag Psi verification
data_X = torch.normal(0,1, [5000,9]).to(device)
data_Y = torch.normal(0,1, [5000,9]).to(device)


Phi = torch.einsum('ij,kj->ik', data_X, data_X)
Psi = torch.einsum('ij,kj->ik', data_Y, data_Y)

trPhiDiagPsi_matmul = torch.trace(
torch.matmul(
    Phi,
    torch.diag(torch.diagonal(Psi))
))
 

trPhiDiagPsi_sumsq = torch.inner(
    torch.sum( torch.square(data_X), axis = 1),
    torch.sum( torch.square(data_Y), axis = 1)
)

print(trPhiDiagPsi_matmul - trPhiDiagPsi_sumsq)
print(trPhiDiagPsi_matmul)
print(trPhiDiagPsi_sumsq)



tensor(0., device='cuda:1')
tensor(406992.8750, device='cuda:1')
tensor(406992.8750, device='cuda:1')


In [42]:
#tr phi tilde psi tilde verification
data_X = torch.normal(0,1, [5000,9]).to(device)
data_Y = torch.normal(0,1, [5000,9]).to(device)

Phi = torch.einsum('ij,kj->ik', data_X, data_X)
Psi = torch.einsum('ij,kj->ik', data_Y, data_Y)
Phi = Phi.fill_diagonal_(0.0)
Psi = Psi.fill_diagonal_(0.0)
tr_phi_tilde_psi_tilde_matmul = torch.trace( torch.matmul(Phi, Psi) )
tr_phi_tilde_psi_tilde_sq = torch.sum(
    torch.square(
        torch.einsum('ji,jk->ik', data_X, data_Y)
    )
)- torch.inner(
    torch.sum( torch.square(data_X), axis = 1),
    torch.sum( torch.square(data_Y), axis = 1)
)
print(tr_phi_tilde_psi_tilde_matmul)
print(tr_phi_tilde_psi_tilde_sq)
print(tr_phi_tilde_psi_tilde_matmul - tr_phi_tilde_psi_tilde_sq)

tensor(-46376.3125, device='cuda:1')
tensor(-46376.3750, device='cuda:1')
tensor(0.0625, device='cuda:1')


In [31]:
    
    
    #matrices
    

    one = torch.ones(n, 1).to(self.cuda_device)
    oneT = torch.transpose(one, 0, 1)

        PhiPsi = torch.matmul(Phi, Psi)
        trPhiPsi = torch.trace(PhiPsi)
        
        onePhiPsiOne = torch.matmul(oneT, torch.matmul(PhiPsi, one))

        onePhione = torch.matmul(oneT, torch.matmul(Phi, one))
        onePsione = torch.matmul(oneT, torch.matmul(Psi, one))
        onePhioneonePsione = torch.matmul(onePhione, onePsione)

 

   Un = 4 * onePhioneonePsione
   - ( 8 * (n-1) ) * onePhiPsiOne
   + ( 4 * (n - 1) * (n - 2) ) * trPhiPsi     
        
    return(Un/n_four)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

In [ ]:
torch.cuda.memory_allocated(device=device)

In [ ]:
X_1 = torch.matmul(data_X, torch.transpose(data_X, 0, 1))
X_2 = torch.einsum('ij,kj->ik', data_X, data_X)
Phi = torch.einsum('ij,kj->ik', data_X, data_X)
    Psi = torch.einsum('ij,kj->ik', data_Y, data_Y)
    Phi = Phi.fill_diagonal_(0.0)
    Psi = Psi.fill_diagonal_(0.0)

term by term comparison

In [171]:

def function1(data_X, data_Y):
    n = data_X.size(dim=0)

    # naive
    Phi = torch.matmul(data_X, torch.transpose(data_X, 0, 1))
    Psi = torch.matmul(data_Y, torch.transpose(data_Y, 0, 1))
    
    Phi = Phi.fill_diagonal_(0.0)
    Psi = Psi.fill_diagonal_(0.0)
    
    n_four = n * (n-1) * (n-2) * (n-3)
    one = torch.ones(n, 1).to(device)
    oneT = torch.transpose(one, 0, 1)

    PhiPsi = torch.matmul(Phi, Psi)
    trPhiPsi = torch.trace(PhiPsi)
    onePhiPsiOne = torch.matmul(oneT, torch.matmul(PhiPsi, one))

    onePhione = torch.matmul(oneT, torch.matmul(Phi, one))
    onePsione = torch.matmul(oneT, torch.matmul(Psi, one))
    onePhioneonePsione = torch.matmul(onePhione, onePsione)
    print(onePhioneonePsione)
    print(onePhiPsiOne)
    print(trPhiPsi)

    Un = 4 * onePhioneonePsione- ( 8 * (n-1) ) * onePhiPsiOne+ ( 4 * (n-1) * (n-2) ) * trPhiPsi     
    print(Un)

    print(4 * onePhioneonePsione
        - ( 8 * (n-1) ) * onePhiPsiOne
        + ( 4 * (n-1) * (n-2) ) * trPhiPsi  )

In [175]:
#data_X = torch.normal(0,1, [5000000,27]).to(device)
#data_Y = torch.normal(0,1, [5000000,27]).to(device)
def function2(data_X, data_Y):
    n = data_X.size(dim = 0)
    n_four = n * (n-1) * (n-2) * (n-3)
    
    X_row_sum = torch.sum(data_X, axis = 0)
    Y_row_sum = torch.sum(data_Y, axis = 0)
    phi_psi = torch.einsum('ji,jk->ik', data_X, data_Y)
    diag_Phi = torch.sum(torch.square(data_X), axis = 1)
    diag_Psi = torch.sum(torch.square(data_Y), axis = 1)
    rowsum_Phi = torch.einsum('i,ji -> j', X_row_sum, data_X)
    rowsum_Psi = torch.einsum('ij, j -> i', data_Y, Y_row_sum)

    #1. one term
    one_Phi_one = torch.inner(X_row_sum, X_row_sum)
    one_Psi_one = torch.inner(Y_row_sum, Y_row_sum)
    
    tr_Phi = torch.sum(torch.square(data_X))
    tr_Psi = torch.sum(torch.square(data_Y))
    
    one_Phi_tilde_one = one_Phi_one - tr_Phi
    one_Psi_tilde_one = one_Psi_one - tr_Psi
    
    onePhioneonePsione = one_Phi_tilde_one * one_Psi_tilde_one


    #2. one one term
    onePhiPsiOne = torch.matmul(
        torch.matmul(X_row_sum, phi_psi),
        Y_row_sum)  + torch.inner(diag_Phi, diag_Psi)-torch.inner(rowsum_Phi, diag_Psi)-torch.inner(diag_Phi, rowsum_Psi)
    
    
    #3. trace term
    trPhiPsi = torch.sum( torch.square(phi_psi) ) - torch.inner(
        torch.sum( torch.square(data_X), axis = 1),
        torch.sum( torch.square(data_Y), axis = 1)
    )

    print(          4 * (onePhioneonePsione - 4 * onePhiPsiOne + 2 * trPhiPsi)
            - 8 * (n-3) *(onePhiPsiOne - trPhiPsi)
            + 4 * (n-3)*(n-2) * trPhiPsi)

    print( 4 * onePhioneonePsione
        - ( 8 * (n-1) ) * onePhiPsiOne
        + ( 4 * (n-1) * (n-2) ) * trPhiPsi  )

In [176]:
data_X = torch.normal(0,1, [500,27]).to(device)
data_Y = torch.normal(0,1, [500,27]).to(device)
function1(data_X, data_Y)
function2(data_X, data_Y)

tensor([[-12613546.]], device='cuda:1')
tensor([[-100186.6094]], device='cuda:1')
tensor(10019.0020, device='cuda:1')
tensor([[1.0308e+10]], device='cuda:1')
tensor([[1.0308e+10]], device='cuda:1')
tensor(1.0308e+10, device='cuda:1')
tensor(1.0308e+10, device='cuda:1')


# Verification of functions

In [257]:
def u_stat_indep_matrix(data_X, data_Y):
    n = data_X.size(dim=0)

    Phi = torch.matmul(data_X, torch.transpose(data_X, 0, 1))
    Psi = torch.matmul(data_Y, torch.transpose(data_Y, 0, 1))
    
    Phi = Phi.fill_diagonal_(0.0)
    Psi = Psi.fill_diagonal_(0.0)
    
    n_four = n * (n-1) * (n-2) * (n-3)
    one = torch.ones(n, 1).to(device)
    oneT = torch.transpose(one, 0, 1)

    PhiPsi = torch.matmul(Phi, Psi)
    trPhiPsi = torch.trace(PhiPsi)
    onePhiPsiOne = torch.matmul(oneT, torch.matmul(PhiPsi, one))

    onePhione = torch.matmul(oneT, torch.matmul(Phi, one))
    onePsione = torch.matmul(oneT, torch.matmul(Psi, one))
    onePhioneonePsione = torch.matmul(onePhione, onePsione)
    
 
    
    Un = (
        4 * (onePhioneonePsione - 4 * onePhiPsiOne + 2 * trPhiPsi)
        - 8 * (n-3) *(onePhiPsiOne - trPhiPsi)
        + 4 * (n-3)*(n-2) * trPhiPsi
    )
    return(Un/n_four)

In [262]:
def u_stat_indep_matrix_efficient( data_X, data_Y):
    #scalars
    n = data_X.size(dim = 0)
    #n_four = n * (n-1) * (n-2) * (n-3)
    log_n_four = (
        torch.log(torch.tensor(n))
        +  
        torch.log(torch.tensor(n-1))
        +
        torch.log(torch.tensor(n-2))
        +
        torch.log(torch.tensor(n-3))
    )
    
    X_row_sum = torch.sum(data_X, axis = 0)
    Y_row_sum = torch.sum(data_Y, axis = 0)
    phi_psi = torch.einsum('ji,jk->ik', data_X, data_Y)
    diag_Phi = torch.sum(torch.square(data_X), axis = 1)
    diag_Psi = torch.sum(torch.square(data_Y), axis = 1)
    rowsum_Phi = torch.einsum('i,ji -> j', X_row_sum, data_X)
    rowsum_Psi = torch.einsum('ij, j -> i', data_Y, Y_row_sum)

    #1. one term
    one_Phi_one = torch.inner(X_row_sum, X_row_sum)
    one_Psi_one = torch.inner(Y_row_sum, Y_row_sum)
    
    tr_Phi = torch.sum(torch.square(data_X))
    tr_Psi = torch.sum(torch.square(data_Y))
    
    one_Phi_tilde_one = one_Phi_one - tr_Phi
    one_Psi_tilde_one = one_Psi_one - tr_Psi
    
    onePhioneonePsione = one_Phi_tilde_one * one_Psi_tilde_one


    #2. one one term
    onePhiPsiOne = torch.matmul(
        torch.matmul(X_row_sum, phi_psi),
        Y_row_sum)  + torch.inner(diag_Phi, diag_Psi)-torch.inner(rowsum_Phi, diag_Psi)-torch.inner(diag_Phi, rowsum_Psi)
    
    
    #3. trace term
    trPhiPsi = torch.sum( torch.square(phi_psi) ) - torch.inner(
        torch.sum( torch.square(data_X), axis = 1),
        torch.sum( torch.square(data_Y), axis = 1)
    )
    
    #print(onePhioneonePsione)
    #print(onePhiPsiOne)
    #print(trPhiPsi)
    
    sums = (
        4 * onePhioneonePsione
    - ( 8 * (n-1) ) * onePhiPsiOne
    + ( 4 * (n-1) * (n-2) ) * trPhiPsi 
    )
    
    Un_sign = torch.sign(sums)
    
    abs_Un = torch.exp(torch.log(torch.abs(sums)) - log_n_four)
    Un = Un_sign * abs_Un
    print(Un)
    
    return(Un)
 

In [201]:
def kernel_indep(fourchunk):
    ip = torch.matmul(fourchunk, torch.transpose(fourchunk, 0, 1))
    return(ip[0,1] + ip[2,3] - ip[0,2] - ip[1,3])

def u_stat_indep_original(data_X, data_Y):
    n = data_X.size(dim = 0)
   # print(f"number of calculation = {2*scipy.misc.comb(n,4) }")
    n_four = n * (n-1) * (n-2) * (n-3)
    U_statistic = 0
    for i in range(n):
        set_j = set(range(n)) - {i}
        for j in set_j:
            set_k = set_j - {j}
            for k in set_k:
                set_r = set_k - {k}
                for r in set_r:
                    comb = [i,j,k,r]
                    U_statistic = U_statistic + (
                        kernel_indep(data_X[comb,]) * kernel_indep(data_Y[comb,])
                        )/n_four
    return(U_statistic)

In [274]:
data_X = torch.normal(0,1, [5,3]).to(device)
data_Y = torch.normal(0,1, [5,3]).to(device)
print(u_stat_indep_matrix(data_X,data_Y))
print(u_stat_indep_matrix_efficient(data_X, data_Y))


tensor([[-3.3775]], device='cuda:1')
tensor(-3.3775, device='cuda:1')
tensor(-3.3775, device='cuda:1')


In [204]:
print(u_stat_indep_original(data_X, data_Y))

tensor(-2.0086, device='cuda:1')


# large dataset validation

In [266]:
data_X = torch.normal(0,1, [100000,27]).to(device)
data_Y = torch.normal(0,1, [100000,27]).to(device)
u_stat_indep_matrix_efficient(data_X, data_Y)


tensor(0.0015, device='cuda:1')


tensor(0.0015, device='cuda:1')

In [213]:
np.log(1000000000000)

27.631021115928547

In [226]:
math.log(100000**4)

46.051701859880914

ddddd

In [275]:
def u_stat_twosample(data, n_1):
    n_2 = data.size(dim = 0) - n_1
    data_x = data[ :n_1, ]
    data_y = data[n_1: , ]
        
    # x only part
    u_x = torch.matmul(data_x, torch.transpose(data_x, 0, 1))
    u_x.fill_diagonal_(0)
    u_x = torch.sum(u_x) / (n_1 * (n_1 - 1))
        
    # y only part
    u_y = torch.matmul(data_y, torch.transpose(data_y, 0, 1))
    u_y.fill_diagonal_(0)
    u_y = torch.sum(u_y) / (n_2 * (n_2 - 1))

    # x, y part
    u_xy = torch.matmul(data_x, torch.transpose(data_y, 0, 1))
    u_xy = torch.sum(u_xy) * ( 2 / (n_1 * n_2) )
    return(u_x + u_y - u_xy)

In [291]:
    

def u_stat_twosample_efficient(data, n_1_int):
    n_1 = torch.tensor(n_1_int)
    n_2 = data.size(dim = 0) - n_1
    data_X = data[ :n_1, ]
    data_Y = data[n_1: , ]
    
    X_row_sum = torch.sum(data_X, axis = 0)
    Y_row_sum = torch.sum(data_Y, axis = 0)
    phi_psi = torch.einsum('ji,jk->ik', data_X, data_Y)

   
    one_Phi_one = torch.inner(X_row_sum, X_row_sum)
    one_Psi_one = torch.inner(Y_row_sum, Y_row_sum)
    
    tr_Phi = torch.sum(torch.square(data_X))
    tr_Psi = torch.sum(torch.square(data_Y))
    
    one_Phi_tilde_one = one_Phi_one - tr_Phi
    one_Psi_tilde_one = one_Psi_one - tr_Psi
    
    onePhioneonePsione = one_Phi_tilde_one * one_Psi_tilde_one
        
    # x only part
    sign_x = torch.sign(one_Phi_tilde_one)
    abs_u_x = torch.exp(torch.log(torch.abs(one_Phi_tilde_one)) - torch.log(n_1) - torch.log(n_1 - 1) )
    u_x = sign_x * abs_u_x
    
            
    # y only part
    sign_y = torch.sign(one_Psi_tilde_one)
    
    abs_u_y = torch.exp(torch.log(torch.abs(one_Psi_tilde_one)) - torch.log(n_2) - torch.log(n_2 - 1) )
    u_y = sign_y * abs_u_y

    # x, y part
    cross = torch.inner(X_row_sum, Y_row_sum)
    sign_cross = torch.sign(cross)
    abs_cross = torch.exp(torch.log(torch.abs(cross)) +torch.log(torch.tensor(2))- torch.log(n_1) - torch.log(n_2) )
    u_cross = sign_cross * abs_cross
    
    return(u_x + u_y - u_cross)

In [296]:
data_X = torch.normal(0,1, [2000,27]).to(device)
n_1 = 1000
print(u_stat_twosample(data_X, n_1))
print(u_stat_twosample_efficient(data_X, n_1))

tensor(0.0060, device='cuda:1')
tensor(0.0060, device='cuda:1')
